In [5]:
import sqlite3

In [6]:
dbname = 'input_data/2021.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

In [7]:
cur.execute("SELECT name FROM sqlite_master WHERE TYPE='table'")
cur.fetchall()

[('pitch_data',),
 ('data_at_bat',),
 ('player',),
 ('game_data',),
 ('batting_stats',),
 ('pitch_stats',)]

In [9]:
cur.execute('select result_big from data_at_bat')
result_big_list = [result[0] for result in cur.fetchall()]
# ngワードが入っているリザルトの除去(本塁打が別れてるのはランニング本塁打があるから)
ng_words = ['けん制', '【代打】', '【代走】', '【守備】', '【継投】', 'ボーク', 'スリーバント', '見逃し', '空振り', '敬遠', '振り逃げ',
            'ファウル', 'ボール', '三振', '四球', '死球', '守備妨害', '打撃妨害', '左本塁打', '中本塁打', '右本塁打']
result_big_list = [result for result in result_big_list if all([ng_word not in result for ng_word in ng_words])]
# 打点の除去
result_big_list = [result.replace('＋1点', '').replace('＋2点', '').replace('＋3点', '').replace('＋4点', '').rstrip() for result in result_big_list]
print(sorted(list(set(result_big_list))))

['', '一2塁打', '一エラー', '一ゴロ', '一フライ', '一ライナー', '一併殺打', '一安打', '一犠打', '一犠打野選', '一野選', '三2塁打', '三エラー', '三ゴロ', '三フライ', '三ライナー', '三併殺打', '三安打', '三犠打', '三犠打野選', '三野選', '中2塁打', '中3塁打', '中エラー', '中ゴロ', '中フライ', '中ランニング本塁打', '中安打', '中犠飛', '二エラー', '二ゴロ', '二フライ', '二ライナー', '二併殺打', '二安打', '二犠打', '二野選', '右2塁打', '右3塁打', '右エラー', '右ゴロ', '右フライ', '右安打', '右犠飛', '左2塁打', '左3塁打', '左エラー', '左ゴロ', '左フライ', '左安打', '左犠飛', '投エラー', '投ゴロ', '投フライ', '投ライナー', '投併殺打', '投安打', '投犠打', '投犠打野選', '投野選', '捕エラー', '捕ゴロ', '捕フライ', '捕併殺打', '捕安打', '捕犠打', '捕犠打野選', '犠打失', '犠飛失', '空振り', '遊2塁打', '遊エラー', '遊ゴロ', '遊フライ', '遊ライナー', '遊併殺打', '遊安打', '遊野選']


In [11]:
cur.execute('select ss from pitch_data')
cur.fetchall()

[('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('1500063',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('700003',),
 ('70000